In [1]:
import pandas as pd
import glob

folder_path = 'ScenarioData/'

csv_files = glob.glob(folder_path + '/*.csv')

df_list = []


for filename in csv_files:
    df = pd.read_csv(filename)
    df.columns = df.columns.str.strip().str.replace(' ', '')
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)

combined_csv_path = folder_path + '\\combined.csv'  
combined_df.to_csv(combined_csv_path, index=False)  

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the data
dafni_impacts_path = 'ScenarioData/combined.csv'
oidj_table_processed_path = 'OiDjTable.csv'

dafni_impacts_df = pd.read_csv(dafni_impacts_path)
oidj_table_processed_df = pd.read_csv(oidj_table_processed_path)

# Data Integration
oidj_table_processed_df.rename(columns={'zonei': 'zone'}, inplace=True)
dafni_impacts_enriched_df = pd.merge(dafni_impacts_df, oidj_table_processed_df, left_on='net_i', right_on='zone', how='left')
dafni_impacts_enriched_df.rename(columns={'Oi_all': 'Oi_net_i', 'Dj_all': 'Dj_net_i'}, inplace=True)
dafni_impacts_enriched_df.drop(['zone'], axis=1, inplace=True)

dafni_impacts_enriched_df = pd.merge(dafni_impacts_enriched_df, oidj_table_processed_df, left_on='net_j', right_on='zone', how='left')
dafni_impacts_enriched_df.rename(columns={'Oi_all': 'Oi_net_j', 'Dj_all': 'Dj_net_j'}, inplace=True)
dafni_impacts_enriched_df.drop(['zone'], axis=1, inplace=True)


dafni_impacts_enriched_df.to_csv('OiDj_merged.csv')

In [3]:
import pandas as pd

# Load the CSV files
oidj_df = pd.read_csv('OiDj_merged.csv')
zonecodes_df = pd.read_csv('EWS_ZoneCodes_xml.csv')

# Perform the merge based on net_i and zonei
merged_df_i = pd.merge(oidj_df, zonecodes_df, left_on='net_i', right_on='zonei', how='left')

# Rename the columns to include 'net_i' in their names to indicate the source of data
columns_to_rename_i = {
    'lat': 'lat_neti',
    'lon': 'lon_neti',
    'osgb36_east': 'osgb36_east_neti',
    'osgb36_north': 'osgb36_north_neti',
    'areakey': 'areakey_neti',
    'zonei': 'zonei_neti',
    'area': 'area_neti'
}
merged_df_i.rename(columns=columns_to_rename_i, inplace=True)

# Perform a second merge based on net_j and zonei
merged_df_j = pd.merge(merged_df_i, zonecodes_df, left_on='net_j', right_on='zonei', how='left')

# Rename the columns from this second merge to include 'net_j' to indicate the source of data
columns_to_rename_j = {
    'lat': 'lat_netj',
    'lon': 'lon_netj',
    'osgb36_east': 'osgb36_east_netj',
    'osgb36_north': 'osgb36_north_netj',
    'areakey': 'areakey_netj',
    'zonei': 'zonei_netj',
    'area': 'area_netj'
}
merged_df_j.rename(columns=columns_to_rename_j, inplace=True)

# Print the head of the merged dataframe to check the result
print(merged_df_j.head())

# Save the merged dataframe to a new CSV file
merged_df_j.to_csv('OiDj_merged.csv', index=False)



   Unnamed: 0  idx      Ck1Road       Ck1Bus      Ck1Rail      Ck2Road  \
0           0    0  7875351.359  3020828.439  9164254.201  7864111.513   
1           1    1  7875351.359  3020828.439  9164254.201  7865522.820   
2           2    2  7875351.359  3020828.439  9164254.201  7863413.383   
3           3    3  7875351.359  3020828.439  9164254.201  7864066.587   
4           4    4  7875351.359  3020828.439  9164254.201  7865096.332   

        Ck2Bus      Ck2Rail    CkDiffRoad    CkDiffBus  ...  osgb36_east_neti  \
0  3018331.319  9177991.167 -11239.845770 -2497.120307  ...          544387.2   
1  3018647.914  9176263.266  -9828.538792 -2180.525344  ...          544387.2   
2  3018177.754  9178842.863 -11937.976120 -2650.685685  ...          544387.2   
3  3018320.702  9178046.710 -11284.771780 -2507.736989  ...          544387.2   
4  3018553.067  9176784.601 -10255.026640 -2275.372523  ...          544387.2   

   osgb36_north_neti  area_neti  areakey_netj  zonei_netj   lat_netj